## BBC - Project
### Data Exploration

In [1]:
DATA_PATH = './data/'

In [13]:
import numpy as np
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records
    
records = load_geo(DATA_PATH + 'GSE21510_family.soft')

The following cell should be executed multiple times to run through the data

In [16]:
nr = next(records)
print(nr)
print('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
print(nr.entity_attributes)

GEO Type: PLATFORM
GEO Id: GPL570
Platform_contact_address: 

Platform_contact_city: Santa Clara

Platform_contact_country: USA

Platform_contact_email: geo@ncbi.nlm.nih.gov, support@affymetrix
.com

Platform_contact_institute: Affymetrix, Inc.

Platform_contact_name: ,,Affymetrix, Inc.

Platform_contact_phone: 888-362-2447

Platform_contact_state: CA

Platform_contact_web_link: http://www.affymetrix.com/index.affx

Platform_contact_zip/postal_code: 95051

Platform_data_row_count: 54675

Platform_description: Affymetrix submissions are typically sub
mitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.g
ov/projects/geo/info/geo_affy.html

Platform_description: 

Platform_description: June 03, 2009: annotation table updated 
with netaffx build 28

Platform_description: June 06, 2012: annotation table updated 
with netaffx build 32

Platform_description: June 23, 2016: annotation table updated 
with netaffx build 35

Platform_distribution: commercial

Platform_g

In [17]:
# re-loading the data
records = load_geo(DATA_PATH + 'GSE21510_family.soft')
series_sample_id = []
sample_titles = []
genes = []
gene_ids = []
nb_cols = 0
nb_rows = 0
data = []
for r in records:
    rea = r.entity_attributes
    if 'Series_geo_accession' in rea:
        if rea['Series_geo_accession'] == 'GSE21510':
            series_sample_id = rea['Series_sample_id']
            nb_cols = len(series_sample_id)
    if 'Sample_title' in rea:
        sample_titles.append(rea['Sample_title'])
        if 'sample_table_begin' in rea:
            nb_rows = rea['Sample_data_row_count'] 
            data.append(r.table_rows)
    if 'platform_table_begin' in rea:
        print('found')
        gene_ids.append(r.table_rows)
                    

found


In [33]:
# the keys of this dict are gene IDs, the values are GB_ACCs
genes_dict = {}
for i, g in enumerate(gene_ids[0]):
    if(i != 0): # do not inclide labels ('ID' & 'GB_ACC')
        genes_dict[g[0]] = g[1]

genes_dict

{'1007_s_at': 'U48705',
 '1053_at': 'M87338',
 '117_at': 'X51757',
 '121_at': 'X69699',
 '1255_g_at': 'L36861',
 '1294_at': 'L13852',
 '1316_at': 'X55005',
 '1320_at': 'X79510',
 '1405_i_at': 'M21121',
 '1431_at': 'J02843',
 '1438_at': 'X75208',
 '1487_at': 'L38487',
 '1494_f_at': 'M33318',
 '1552256_a_at': 'NM_005505',
 '1552257_a_at': 'NM_015140',
 '1552258_at': 'NM_052871',
 '1552261_at': 'NM_080735',
 '1552263_at': 'NM_138957',
 '1552264_a_at': 'NM_138957',
 '1552266_at': 'NM_145004',
 '1552269_at': 'NM_138796',
 '1552271_at': 'NM_153359',
 '1552272_a_at': 'NM_153359',
 '1552274_at': 'BC014479',
 '1552275_s_at': 'BG573647',
 '1552276_a_at': 'NM_080432',
 '1552277_a_at': 'NM_080655',
 '1552278_a_at': 'NM_080669',
 '1552279_a_at': 'AK074161',
 '1552280_at': 'NM_138379',
 '1552281_at': 'NM_173596',
 '1552283_s_at': 'NM_024786',
 '1552286_at': 'NM_080653',
 '1552287_s_at': 'NM_001132',
 '1552288_at': 'BC034926',
 '1552289_a_at': 'BC034926',
 '1552291_at': 'NM_017861',
 '1552293_at': 'N

data will become a 2d numpy array of shape (54676, 149) == (NB_GENES+1, NB_SAMPLES+1).  
First row will contain the sample ids, first column will contain the ID_REFs of the genes.

In [5]:
all_data = np.ndarray((int(nb_rows)+1, int(nb_cols)+1), dtype=object)
# labels
all_data[0, 0] = 'ID_REF'
all_data[0, 1:] = np.array(series_sample_id)

for i, d in enumerate(data):
    values = np.array(d[1:])
    if (i == 0):
        all_data[1:, 0] = values[:, 0]
    all_data[1:, i+1] = values[:, 1]
data = np.array(all_data)

In [6]:
data.shape

(54676, 149)

In [7]:
print(series_sample_id)
print('')
print('')
print(sample_titles)
print('')
print(len(data), len(data[0]))
print(data[1][0])

['GSM537330', 'GSM537331', 'GSM537332', 'GSM537333', 'GSM537334', 'GSM537335', 'GSM537336', 'GSM537337', 'GSM537338', 'GSM537339', 'GSM537340', 'GSM537341', 'GSM537342', 'GSM537343', 'GSM537344', 'GSM537345', 'GSM537346', 'GSM537347', 'GSM537348', 'GSM537349', 'GSM537350', 'GSM537351', 'GSM537352', 'GSM537353', 'GSM537354', 'GSM537355', 'GSM537356', 'GSM537357', 'GSM537358', 'GSM537359', 'GSM537360', 'GSM537361', 'GSM537362', 'GSM537363', 'GSM537364', 'GSM537365', 'GSM537366', 'GSM537367', 'GSM537368', 'GSM537369', 'GSM537370', 'GSM537371', 'GSM537372', 'GSM537373', 'GSM537374', 'GSM537375', 'GSM537376', 'GSM537377', 'GSM537378', 'GSM537379', 'GSM537380', 'GSM537381', 'GSM537382', 'GSM537383', 'GSM537384', 'GSM537385', 'GSM537386', 'GSM537387', 'GSM537388', 'GSM537389', 'GSM537390', 'GSM537391', 'GSM537392', 'GSM537393', 'GSM537394', 'GSM537395', 'GSM537396', 'GSM537397', 'GSM537398', 'GSM537399', 'GSM537400', 'GSM537401', 'GSM537402', 'GSM537403', 'GSM537404', 'GSM537405', 'GSM537406'

NameError: name 'samples_geo_accessions' is not defined

In [ ]:
import re

samples = {}
patient_names = []
for i_t, title in enumerate(sample_titles):
    split = title.split(',')
    if(re.search('cancer', title)):
        samples[series_sample_id[i_t]] = 'cancer'
    else:
        samples[series_sample_id[i_t]] = 'normal'
        
    patient_names.append(split[0])
        
samples

In [ ]:
print(sample_titles[104])